In [1]:
import xarray as xr
import numpy as np
from dask.distributed import Client, LocalCluster
import torch
import xarray as xr

cluster=  LocalCluster(n_workers=4, memory_limit='20GiB')
client = Client(cluster)
client

/felles_gfi/felles_gfi_users/rogui7909/miniforge3/envs/robin/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 40099 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:40099/status,
Dashboard: http://127.0.0.1:40099/status,Workers: 4
Total threads: 72,Total memory: 80.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45094,Workers: 4
Dashboard: http://127.0.0.1:40099/status,Total threads: 72
Started: Just now,Total memory: 80.00 GiB
Comm: tcp://127.0.0.1:38462,Total threads: 18
Dashboard: http://127.0.0.1:45621/status,Memory: 20.00 GiB
Nanny: tcp://127.0.0.1:39961,


In [2]:
era5 = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/1959-2022-6h-512x256_equiangular_conservative.zarr',
    chunks=dict(lon=50, lat=50, time=4*120, level=1),
    storage_options=dict(token='anon'),
)

/tmp/ipykernel_296411/550085691.py:1: UserWarning: The specified chunks separate the stored chunks along dimension "level" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  era5 = xr.open_zarr(


In [31]:
precip = era5['total_precipitation_6hr']*1000
precip_out = precip.sel(latitude=slice(0,90)).resample(time='D').sum(keep_attrs=True).rename('pr').to_dataset()
precip_out['mean_pr'] = precip_out.pr.weighted(np.cos(np.deg2rad(precip_out.latitude))).mean(['longitude','latitude','time'])
precip_out['std_pr'] = precip_out.pr.weighted(np.cos(np.deg2rad(precip_out.latitude))).std(['longitude','latitude','time'])

precip_out.assign_coords(longitude=(((precip_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/pr.nc')


In [32]:
tcwv = era5['total_column_water_vapour']
tcwv_out = tcwv.sel(latitude=slice(0,90)).resample(time='D').mean(keep_attrs=True).rename('tcwv').to_dataset()
tcwv_out['mean_tcwv'] = tcwv_out.tcwv.weighted(np.cos(np.deg2rad(tcwv_out.latitude))).mean(['longitude','latitude','time'])
tcwv_out['std_tcwv'] = tcwv_out.tcwv.weighted(np.cos(np.deg2rad(tcwv_out.latitude))).std(['longitude','latitude','time'])

tcwv_out.assign_coords(longitude=(((tcwv_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/tcwv.nc')

In [29]:
uwind = era5['u_component_of_wind'].sel(level=850, drop=True).rename('u850')
uwind_out = uwind.sel(latitude=slice(0,90)).resample(time='D').mean(keep_attrs=True).rename('u850').to_dataset()
uwind_out['mean_u850'] = uwind_out.u850.weighted(np.cos(np.deg2rad(uwind_out.latitude))).mean(['longitude','latitude','time'])
uwind_out['std_u850'] = uwind_out.u850.weighted(np.cos(np.deg2rad(uwind_out.latitude))).std(['longitude','latitude','time'])

uwind_out.assign_coords(longitude=(((uwind_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/u850.nc')

2025-02-13 12:46:12,249 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.66 GiB -- Worker memory limit: 8.00 GiB
2025-02-13 12:46:14,449 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.56 GiB -- Worker memory limit: 8.00 GiB
2025-02-13 12:46:15,983 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [30]:
vwind = era5['v_component_of_wind'].sel(level=850, drop=True).rename('v850')
vwind_out = vwind.sel(latitude=slice(0,90)).resample(time='D').mean(keep_attrs=True).rename('v850').to_dataset()
vwind_out['mean_v850'] = vwind_out.v850.weighted(np.cos(np.deg2rad(vwind_out.latitude))).mean(['longitude','latitude','time'])
vwind_out['std_v850'] = vwind_out.v850.weighted(np.cos(np.deg2rad(vwind_out.latitude))).std(['longitude','latitude','time'])

vwind_out.assign_coords(longitude=(((vwind_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/v850.nc')

2025-02-13 12:55:05,337 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.63 GiB -- Worker memory limit: 8.00 GiB
2025-02-13 12:55:08,092 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 5.63 GiB -- Worker memory limit: 8.00 GiB
2025-02-13 12:55:18,144 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- U

In [22]:
z500 = era5['geopotential'].sel(level=850, drop=True).rename('z500')
z500_out = z500.sel(latitude=slice(0,90)).resample(time='D').mean(keep_attrs=True).rename('z500').to_dataset()
z500_out['mean_z500'] = z500_out.z500.weighted(np.cos(np.deg2rad(z500_out.latitude))).mean(['longitude','latitude','time'])
z500_out['std_z500'] = z500_out.z500.weighted(np.cos(np.deg2rad(z500_out.latitude))).std(['longitude','latitude','time'])

z500_out.assign_coords(longitude=(((z500_out.longitude + 180) % 360) - 180)).sortby('longitude').to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/z500.nc')

# make tensor

In [3]:

process_variables = dict(z500='standard',
                         u850='std_only',
                         v850='std_only',
                         tcwv='standard',
                         pr='standard'
                            )

In [4]:
variables = ['z500','u850','v850','tcwv','pr']
lon_extent = slice(-90,89.5)
lat_extent = slice(20,90)

print('loading data ...')
all_data_in = dict()
for variable in variables:
    with xr.open_dataset(f'/Data/gfi/users/rogui7909/data/ERA5/{variable}.nc', chunks = dict(longitude=50, latitude=50, time=365*10)) as ds:
        data_ = ds.sel(latitude=lat_extent, longitude=lon_extent)[variable]
        time = ds.time
        if process_variables[variable] == 'standard':
            data_ = (data_ - ds[f'mean_{variable}'].values)/ds[f'std_{variable}'].values
        elif process_variables[variable] == 'std_only':
            data_ = (data_)/ds[f'std_{variable}'].values
        all_data_in[variable]=data_
        print('   --> ',variable,'loaded')
data_in = xr.Dataset(all_data_in).to_array('var_name')
# data_in = data_in.transpose(1,0,2,3)
print('Input data ready')

# tensor_in = torch.Tensor(data_in).type(torch.float32)

loading data ...
   -->  z500 loaded
   -->  u850 loaded
   -->  v850 loaded
   -->  tcwv loaded
   -->  pr loaded
Input data ready


In [5]:
data_numpy = data_in.transpose('time','var_name','latitude','longitude').values

# to_netcdf('/Data/gfi/users/rogui7909/data/ERA5/era5_in_data.nc')

In [ ]:
np.save( "/Data/gfi/users/rogui7909/data/ERA5/numpy_era5_in.npy", data_numpy)

2025-02-14 12:13:55,081 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/felles_gfi/felles_gfi_users/rogui7909/miniforge3/envs/robin/lib/python3.12/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/felles_gfi/felles_gfi_users/rogui7909/miniforge3/envs/robin/lib/python3.12/site-packages/distributed/worker.py", line 1269, in heartbeat
    response = await retry_operation(
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/felles_gfi/felles_gfi_users/rogui7909/miniforge3/envs/robin/lib/python3.12/site-packages/distributed/utils_comm.py", line 441, in retry_operation
    return await retry(
           ^^^^^^^^^^

In [1]:
import torch

In [2]:
tensor_in = torch.load("/Data/gfi/users/rogui7909/data/ERA5/tensor_era5_in.pt", weights_only=True)

In [6]:
tensor_in.transpose(1,0,2,3)

TypeError: transpose() received an invalid combination of arguments - got (int, int, int, int), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)
